In [1]:
import sys; sys.path.append("../")

import pandas as pd

from clean.post_pprr import clean
from lib.uid import gen_uid
from lib.date import combine_date_columns
from lib.match import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)

In [2]:
post = clean()
post

../lib/clean.py:152: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/post_pprr.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, "agency"] = df.agency.str.strip()\


,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,uid
0,14th jdc da office,abshire,dustin,full-time,1998-06-11,2010-10-27,2013,06,01,43ee7cb8a15ca49e883372d3c99988e1
1,17th da office,ourso,damian,full-time,2004-05-13,2010-10-07,2008,11,11,8875f7c0749a1de0923c1242a4363622
2,17th da office,besson,bruce,full-time,NaT,2008-01-26,1997,02,11,2c35336b17f9986c7547656bb72508fe
3,17th da office,bourg,billy,full-time,1991-04-24,2010-10-07,1990,10,12,e17ebe5f0a893d9cb6aac11bf9673c32
4,19th da office,cadarette,steve,full-time,2003-08-28,2020-04-28,2016,04,11,84a22650eb29e5d7d17721d59c3a2fc6
...,...,...,...,...,...,...,...,...,...,...
38898,youngsville pd,fruge,hilary,reserve,2010-11-18,2020-06-22,2017,11,27,2e785e632588b75ebe30c3651f80837c
38899,youngsville pd,mouton,todd,reserve,NaT,2020-06-15,2017,07,01,e2642ae3d8360270ac3b4cfd5d8c5ea4
38900,youngsville pd,hidalgo,jeremy,reserve,NaT,2020-07-31,2017,12,01,a5ca9eebadff5a86ea4e7dd507ba1f73
38901,zachary pd,viccellio,david,reserve,NaT,2019-12-04,2019,12,03,3c6043ea0d02ebbab16968c88c9107f0


In [5]:
post.loc[post.agency == 'e. baton rouge so']

,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,uid
7062,e. baton rouge so,lochridge,joseph,full-time,2006-02-02,2010-02-03,2008,07,16,384e1140886594a967d9362ed0a48523
7063,e. baton rouge so,gaines,sean,full-time,2007-10-04,2010-05-11,2007,03,07,bdb12aad107f764fd411262fa1f9a902
7064,e. baton rouge so,taylor,doris,full-time,2013-11-08,2020-05-29,2019,08,06,762838a777e17c66cc74daad0581a224
7065,e. baton rouge so,roberts,mark,full-time,1991-11-07,2020-02-19,2007,12,17,91bdef70024702ec57ac1f15a97a29d7
7066,e. baton rouge so,walsh,thomas,full-time,2008-06-26,2020-06-02,2007,10,10,eae958be2c9f3cd7e10643c5b27e6108
...,...,...,...,...,...,...,...,...,...,...
37099,e. baton rouge so,james,anthony,reserve,NaT,NaT,2019,12,06,73f69763fc5dbecf27fb0e7c70339659
37100,e. baton rouge so,johnson,rayland,reserve,NaT,2020-03-07,2020,02,13,a1bb1c11febdba0956b3fadc4071d17d
37101,e. baton rouge so,crawford,cody,reserve,NaT,2020-03-07,2019,12,06,cbad8477f9dbedf6fde0e816a048a148
37102,e. baton rouge so,sharer,tucker,reserve,NaT,2020-03-07,2019,12,06,22aed5b5cc10f839853e75c7d32e7f4c


In [11]:
post.loc[(post.last_pc_12_qualification_date == '2010-05-06') & (post.agency == 'baton rouge pd')]

,agency,last_name,first_name,employment_status,level_1_cert_date,last_pc_12_qualification_date,hire_year,hire_month,hire_day,uid
1879,baton rouge pd,sansone,david,full-time,NaT,2010-05-06,1980,12,28,8d668af729f465681a6c830b8761a80b
1882,baton rouge pd,gipson,jewel,full-time,NaT,2010-05-06,1977,04,01,2119a96103d7832cd58a2f3805a5deee
1922,baton rouge pd,davis,victoria,full-time,NaT,2010-05-06,1981,05,12,3bc918fe8cfa282234633f4e88a24611
2190,baton rouge pd,davis,james,full-time,NaT,2010-05-06,1985,03,25,0c34b2655c77fcb9be681a266bd941b2
2204,baton rouge pd,cody,michael,full-time,NaT,2010-05-06,1985,06,17,25cffb9ead8f8dacca16577c66d81d0e
